In [48]:
import os
import time
import json
import datetime
import numpy as np
import pandas as pd
import pickle
from datetime import datetime

In [10]:
import seaborn as sns
from plotly import express as px

In [4]:
%%time
submit = json.load(open('./data/sample_submission/sample_submission.json', 'r', encoding='utf8')) 
df_dic = pickle.load(open('./mydata/df_dic.plk','rb'))
plc_lst = pickle.load(open('./mydata/plc_lst.plk','rb'))
fct_lst = ['pH', 'COD', 'SS', 'N', 'P', 'T']

CPU times: user 839 ms, sys: 116 ms, total: 955 ms
Wall time: 955 ms


In [ ]:
''' 전체 통채로 만들기 '''
plc_sorted_lst = sorted(plc_lst)

dfs = []
for lo in plc_sorted_lst:
    df_temp = df_dic[lo]
    df_temp['loc'] = lo
    dfs.append(df_temp)
dfall = pd.concat(dfs)

In [44]:
''' 기록이 시작되는 공통 날짜 구하기 '''
dates = []
for lo in plc_sorted_lst:
    df_temp = df_dic[lo]
    dates.append(df_temp.ds[0])
mx_date= max(dates)
print(mx_date)  

20150423


In [50]:
''' 최소 날짜가 포함되어 있는지 확인. -> 모두 포함되어 있다. '''
for lo in plc_sorted_lst:
    df_temp = df_dic[lo]
    if ~(df_temp.ds == mx_date).any():
        print(lo)

In [73]:
''' datetime  열 만들기 '''
dt_list =  [ datetime.strptime(x, "%Y%m%d")for x in  list(dfall['ds'])]
dfall['dst'] = dt_list

''' 공통 날짜로 맞추기 '''
dfall_comm = dfall.loc[dfall.dst >= datetime(2015,4,23)]
dfall_comm = dfall_comm.reset_index(drop=True)

In [74]:
''' 모든 업장의 데이터 길이가 같은지 확인 -> 1015 개로 똑같다. '''
res = dfall_comm.groupby(by = 'loc').size()

In [84]:
for col in ['pH', 'COD', 'SS', 'N', 'P', 'T']:
    dfall_comm[col] = pd.to_numeric(dfall_comm[col])

In [86]:
pickle.dump(dfall_comm , open('./mydata/dfall_comm.plk','wb'))

---
## 6개의 피쳐끼리 corr 보기 

In [87]:
dfall_comm.columns

Index(['ds', 'pH', 'COD', 'SS', 'N', 'P', 'T', 'loc', 'dst'], dtype='object')

In [88]:
df_corr = dfall_comm[['pH', 'COD', 'SS', 'N', 'P', 'T']]

In [89]:
df_corr.corr('pearson')

,pH,COD,SS,N,P,T
pH,1.000000,0.030939,-0.012424,-0.037054,0.019180,0.004092
COD,0.030939,1.000000,0.157425,0.131255,0.101528,0.008062
SS,-0.012424,0.157425,1.000000,0.116247,0.094190,0.000653
N,-0.037054,0.131255,0.116247,1.000000,0.123518,-0.007270
P,0.019180,0.101528,0.094190,0.123518,1.000000,0.011217
T,0.004092,0.008062,0.000653,-0.007270,0.011217,1.000000


---
## 시각화로 확인

In [65]:
dfall_comm.columns

Index(['index', 'ds', 'pH', 'COD', 'SS', 'N', 'P', 'T', 'loc', 'dst'], dtype='object')

In [63]:
temp = dfall_comm.iloc[:1015*5]

In [ ]:
fig = px.line(temp, x="ds", y="T", color='loc')
fig.show()

In [ ]:
for lo in plc_sorted_lst:
    df_temp = df_dic[lo]
    print(df_temp.ds[0],df_temp.iloc[-1,:].ds)

In [28]:
dfs = []
for lo in plc_sorted_lst:
    df_temp = df_dic[lo]
    df_temp['loc'] = lo
    dfs.append(df_temp)

In [22]:
dfall = pd.concat(dfs)

In [ ]:

fig = px.line(dfall, x="ds", y="pH", color='loc')
fig.show()